# Modelo Regresión Logística Regularizada con NumPy (sin sklearn)

In [1]:
#aqui ya no se explica mucho todo lo realizado porque se trato igual la data como en el de arboles de decisión
#importamos las librerias necesarias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import joblib

In [2]:
# Cargamos los datos
datos = pd.read_csv("train.csv")

In [3]:
#llenamos con el promedio las edades faltantes
#borramos columnas que no nos aportan nada al modelo
datos["Age"] = datos["Age"].fillna(datos["Age"].mean())
datos = datos.drop(["Cabin"], axis=1)
datos = datos.dropna()
datos = datos.drop(["Name", "PassengerId", "Ticket"], axis=1)


In [4]:
#pasamos a numericas las variables categóricas
dummies_sex = pd.get_dummies(datos["Sex"], drop_first=True)
datos = datos.join(dummies_sex)
datos = datos.drop(["Sex"], axis=1)
dummies_embarked = pd.get_dummies(datos["Embarked"], drop_first=True)
datos = datos.join(dummies_embarked)
datos = datos.drop(["Embarked"], axis=1)

### MODELO MANUAL DE REGRESION LOGISTICA REGULARIZADA

In [6]:

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# trabajamos con los datos que ya estaban procesados
X = datos.drop("Survived", axis=1).values.astype(np.float64)
y = datos["Survived"].values.astype(np.float64)

# dividimos datos de entrenamiento y pruebas 80% y 20%  y el random_state es para que podamos replicar siempre la misma separación
X_ent, X_pru, y_ent, y_pru = train_test_split(X, y, test_size=0.2, random_state=42)

# Agregar columna de 1s para el sesgo (bias)
X_ent = np.c_[np.ones((X_ent.shape[0], 1)), X_ent]
X_pru = np.c_[np.ones((X_pru.shape[0], 1)), X_pru]

# Inicializar parámetros
theta = np.zeros(X_ent.shape[1], dtype=np.float64)

# Función sigmoide
def sigmoid(z):
    z = np.array(z, dtype=np.float64)
    return 1 / (1 + np.exp(-z))

# Función de pérdida con regularización L2
def compute_loss(X, y, theta, lambda_):
    m = len(y)
    h = sigmoid(X @ theta)
    loss = -np.mean(y * np.log(h + 1e-15) + (1 - y) * np.log(1 - h + 1e-15))
    reg = (lambda_ / (2 * m)) * np.sum(theta[1:] ** 2)  # No regularizar el bias
    return loss + reg

# Descenso del gradiente
def gradient_descent(X, y, theta, alpha, lambda_, epochs):
    m = len(y)
    for i in range(epochs):
        h = sigmoid(X @ theta)
        gradient = (1 / m) * (X.T @ (h - y))
        gradient[1:] += (lambda_ / m) * theta[1:]  
        theta -= alpha * gradient
    return theta

# Entrenammos el modelo con los siguientes parametros
lambda_ = 1.0
alpha = 0.1
epochs = 1000
theta = gradient_descent(X_ent, y_ent, theta, alpha, lambda_, epochs)

# hacemos nuestra función de prediccion
def predecir(X, theta):
    return (sigmoid(X @ theta) >= 0.5).astype(int)

y_pred = predecir(X_pru, theta)

# Evaluar
print("Accuracy  o precisión:", accuracy_score(y_pru, y_pred))


Accuracy  o precisión: 0.7078651685393258
